In [2]:
import psycopg2
import pandas as pd

/home/lmoraes/miniconda/envs/machine_teaching/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
connection = psycopg2.connect(user = "machineteaching",
                                  password = "***REMOVED***",
                                  host = "localhost",
#                                   port = "5432",
                                  database = "machineteaching")
connection.autocommit=True
cursor = connection.cursor()

In [300]:
create_table_query = '''CREATE TABLE EXPERIMENTS_SOLUTION
    (EXPERIMENT_ID SERIAL PRIMARY KEY,
    DATASET TEXT NOT NULL,
    X REAL ARRAY NOT NULL,
    Y REAL ARRAY NOT NULL,
    VECTORIZER TEXT NOT NULL,
    IS_BINARY BOOLEAN NOT NULL,
    MIN_DF REAL NOT NULL,
    DISTANCE TEXT NOT NULL,
    K INTEGER NOT NULL,
    METHOD TEXT NOT NULL,
    MODEL  bytea NOT NULL,
    CLUSTERING_TIME INTEGER NOT NULL,
    GAP REAL NOT NULL,
    GAP_STD REAL NOT NULL,
    GAP_TIME REAL NOT NULL,
    SILHOUETTE REAL NOT NULL,
    SILHOUETTE_SAMPLES REAL ARRAY NOT NULL,
    SILHOUETTE_TIME REAL NOT NULL,
    COHERENCE_SAMPLES5 REAL ARRAY NOT NULL,
    COHERENCE_SAMPLES10 REAL ARRAY NOT NULL,
    COHERENCE_SAMPLES15 REAL ARRAY NOT NULL,
    COHERENCE_MED5 REAL NOT NULL,
    COHERENCE_MED10 REAL NOT NULL,
    COHERENCE_MED15 REAL NOT NULL,
    COHERENCE_STD5 REAL NOT NULL,
    COHERENCE_STD10 REAL NOT NULL,
    COHERENCE_STD15 REAL NOT NULL,
    COHERENCE_TIME REAL NOT NULL,
    COHERENCE_K5 INTEGER NOT NULL,
    COHERENCE_K10 INTEGER NOT NULL,
    COHERENCE_K15 INTEGER NOT NULL)
    '''

cursor.execute(create_table_query)
connection.commit()

In [301]:
cols = ['dataset', 'k', 'min_df', 'is_binary', 'vectorizer', 'method', 'distance', 'gap', 'clustering_time', 'gap_time', 'silhouette_time', 'coherence_time']
select_query = '''SELECT %s FROM EXPERIMENTS_SOLUTION''' % ", ".join(cols)
cursor.execute(select_query)
r = cursor.fetchall()
# print(r)

In [302]:
df = pd.DataFrame(r, columns=cols)

In [303]:
df['total_time'] = df.sum(axis=1)

In [304]:
df[(df['dataset'] == 'solution_all')].sort_values(['k','min_df', 'is_binary','vectorizer'])
#    & (df['vectorizer'] == 'TfidfVectorizer') & (df['is_binary'] == True) & (df['distance'] == 'euclidean')
# df

,dataset,k,min_df,is_binary,vectorizer,method,distance,gap,clustering_time,gap_time,silhouette_time,coherence_time,total_time
23,solution_all,2,0.05,False,CountVectorizer,nmf,euclidean,0.448773,0,2.912770,0.013515,0.030507,5.455565
39,solution_all,2,0.05,False,CountVectorizer,lda,euclidean,0.434688,1,28.782500,0.011793,0.082940,32.361921
0,solution_all,2,0.05,True,CountVectorizer,nmf,euclidean,-0.028890,0,3.816770,0.042967,0.028983,6.909830
14,solution_all,2,0.05,True,CountVectorizer,lda,euclidean,-0.009405,2,19.196400,0.007849,0.047179,24.292023
81,solution_all,2,0.10,False,CountVectorizer,nmf,euclidean,0.370334,0,1.864980,0.002785,0.057495,4.395594
94,solution_all,2,0.10,False,CountVectorizer,lda,euclidean,0.352009,1,24.975100,0.001847,0.008978,28.437934
52,solution_all,2,0.10,True,CountVectorizer,nmf,euclidean,-0.296475,0,1.924240,0.011591,0.021199,4.760555
58,solution_all,2,0.10,True,CountVectorizer,lda,euclidean,-0.269435,1,12.348500,0.001107,0.004528,16.184700
19,solution_all,2,0.15,False,CountVectorizer,lda,euclidean,0.320345,1,26.271800,0.002813,0.012566,29.757524
136,solution_all,2,0.15,False,CountVectorizer,nmf,euclidean,0.359416,0,1.232880,0.029913,0.007339,3.779548


In [293]:
# c5_df[['min_df', 'is_binary', 'vectorizer', 'method', 'distance', 'k']].drop_duplicates()

,min_df,is_binary,vectorizer,method,distance,k
0,0.05,True,TfidfVectorizer,lda,euclidean,13
1,0.05,True,TfidfVectorizer,lda,euclidean,11
2,0.35,True,NCutVectorizer,nmf,euclidean,7
3,0.40,True,CountVectorizer,nmf,euclidean,13
4,0.50,False,NCutVectorizer,nmf,euclidean,12
5,0.45,False,NCutVectorizer,nmf,euclidean,15
6,0.50,False,CountVectorizer,nmf,euclidean,11
7,0.05,True,CountVectorizer,nmf,euclidean,15
8,0.20,False,CountVectorizer,nmf,euclidean,15
9,0.05,True,CountVectorizer,lda,euclidean,13


In [299]:
drop_query = '''DROP TABLE EXPERIMENTS_SOLUTION CASCADE'''
cursor.execute(drop_query)
connection.commit()

In [295]:
create_results_table = '''CREATE TABLE EXPERIMENTS_SOLUTION_RESULT
    (EXPERIMENT_ID INTEGER REFERENCES EXPERIMENTS(EXPERIMENT_ID),
    GAP_K INTEGER NOT NULL,
    GAP_K2 INTEGER NOT NULL,
    SILHOUETTE_K INTEGER NOT NULL,
    SILHOUETTE_K2 INTEGER NOT NULL,
    COHERENCE_K5 INTEGER NOT NULL,
    COHERENCE_K2_5 INTEGER NOT NULL,
    COHERENCE_K10 INTEGER NOT NULL,
    COHERENCE_K2_10 INTEGER NOT NULL,
    COHERENCE_K15 INTEGER NOT NULL,
    COHERENCE_K2_15 INTEGER NOT NULL)
    '''

cursor.execute(create_results_table)
connection.commit()

In [296]:
alter_results_table = """ALTER TABLE EXPERIMENTS_SOLUTION_RESULT
 ADD CONSTRAINT UC_SOLUTION_RESULT UNIQUE (EXPERIMENT_ID)"""

cursor.execute(alter_results_table)
connection.commit()

In [11]:
# cols_result = ["gap_k", "gap_k2", "silhouette_k", "silhouette_k2", "coherence_k", "coherence_k2"]
# cols_result_sql = ["experiments_result.%s" % item for item in cols_result]
cols_experiment = ['experiment_id', 'k', 'dataset', 'min_df', 'is_binary', 'vectorizer', 'method', 'distance', 'silhouette', 'coherence_med5', 'coherence_med10','coherence_med15']
cols_experiment_sql = ["experiments_solution.%s" % item for item in cols_experiment]
# select_query = '''SELECT %s FROM EXPERIMENTS where dataset = 'solution_cleaned' order by silhouette desc  '''  % ", ".join(cols_experiment_sql)
# select_query = '''SELECT %s FROM EXPERIMENTS_SOLUTION where dataset = 'solution_all' and k <= 10 order  by coherence_med5 desc, coherence_med10 desc, coherence_med15 desc   '''  % ", ".join(cols_experiment_sql)
# select_query = '''SELECT %s FROM EXPERIMENTS_SOLUTION where dataset = 'solution_all' and k <= 10 order by coherence_med10 desc,  coherence_med5 desc, coherence_med15 desc  '''  % ", ".join(cols_experiment_sql)
select_query = '''SELECT %s FROM EXPERIMENTS_SOLUTION where dataset = 'solution_all' and k <= 10 order by coherence_med15 desc, coherence_med5 desc, coherence_med10 desc  '''  % ", ".join(cols_experiment_sql)
cursor.execute(select_query)
r = cursor.fetchall()
# print(r)

In [12]:
c15_df = pd.DataFrame(r, columns=cols_experiment)

In [29]:
c15_df[c15_df['experiment_id'] == 10]

,experiment_id,k,dataset,min_df,is_binary,vectorizer,method,distance,silhouette,coherence_med5,coherence_med10,coherence_med15
11,10,9,solution_all,0.05,True,CountVectorizer,nmf,euclidean,0.138364,0.001998,-0.073664,-0.080727


In [7]:
c5_df

,experiment_id,k,dataset,min_df,is_binary,vectorizer,method,distance,silhouette,coherence_med5,coherence_med10,coherence_med15
0,1136,8,solution_all,0.05,True,NCutVectorizer,lda,euclidean,0.070547,NaN,0.004988,-0.319701
1,1463,7,solution_all,0.35,True,NCutVectorizer,nmf,euclidean,-0.046570,0.006639,0.006639,-0.110839
2,574,6,solution_all,0.05,True,TfidfVectorizer,lda,euclidean,0.057159,0.004988,-0.211332,-0.373820
3,150,10,solution_all,0.15,False,CountVectorizer,nmf,euclidean,0.037232,0.002912,-0.130937,-0.162715
4,349,9,solution_all,0.35,True,CountVectorizer,nmf,euclidean,0.028150,0.002732,0.002732,-0.373123
5,1522,8,solution_all,0.40,True,NCutVectorizer,nmf,euclidean,0.040625,0.002732,-0.088308,-0.424627
6,1470,10,solution_all,0.35,True,NCutVectorizer,nmf,euclidean,0.054895,0.002497,-0.198680,-0.415599
7,543,10,solution_all,0.50,False,CountVectorizer,nmf,euclidean,-0.128285,0.002497,-0.242574,-0.408898
8,1525,9,solution_all,0.40,True,NCutVectorizer,nmf,euclidean,0.075235,0.002497,-0.259428,-0.889086
9,233,9,solution_all,0.25,True,CountVectorizer,nmf,euclidean,0.033335,0.002289,-0.036604,-0.362066


In [10]:
c10_df

,experiment_id,k,dataset,min_df,is_binary,vectorizer,method,distance,silhouette,coherence_med5,coherence_med10,coherence_med15
0,1463,7,solution_all,0.35,True,NCutVectorizer,nmf,euclidean,-0.046570,0.006639,0.006639,-0.110839
1,1136,8,solution_all,0.05,True,NCutVectorizer,lda,euclidean,0.070547,NaN,0.004988,-0.319701
2,349,9,solution_all,0.35,True,CountVectorizer,nmf,euclidean,0.028150,0.002732,0.002732,-0.373123
3,1527,10,solution_all,0.40,True,NCutVectorizer,nmf,euclidean,-0.007969,0.002219,0.002219,-0.539140
4,409,8,solution_all,0.40,True,CountVectorizer,lda,euclidean,0.176273,0.001428,0.001428,-0.075124
5,401,9,solution_all,0.40,True,CountVectorizer,nmf,euclidean,0.100853,0.001098,0.001098,-0.090525
6,348,6,solution_all,0.35,True,CountVectorizer,lda,euclidean,0.127819,0.000999,0.000999,-0.162939
7,343,4,solution_all,0.35,True,CountVectorizer,lda,euclidean,0.127819,0.000999,0.000999,-0.162939
8,353,7,solution_all,0.35,True,CountVectorizer,lda,euclidean,0.128210,0.000909,0.000909,-0.139640
9,296,7,solution_all,0.30,True,CountVectorizer,lda,euclidean,0.230573,0.000909,0.000833,-0.086429


In [13]:
c15_df

,experiment_id,k,dataset,min_df,is_binary,vectorizer,method,distance,silhouette,coherence_med5,coherence_med10,coherence_med15
0,133,9,solution_all,0.15,True,CountVectorizer,lda,euclidean,0.159251,0.001428,-0.029931,-0.044517
1,122,5,solution_all,0.15,True,CountVectorizer,lda,euclidean,0.139090,0.000666,-0.049023,-0.048802
2,21,9,solution_all,0.05,True,CountVectorizer,lda,euclidean,0.067715,0.000833,-0.037943,-0.052919
3,246,10,solution_all,0.25,True,CountVectorizer,lda,euclidean,0.192126,0.001010,-0.022226,-0.054641
4,301,9,solution_all,0.30,True,CountVectorizer,lda,euclidean,0.189393,0.001079,-0.012671,-0.055538
5,240,7,solution_all,0.25,True,CountVectorizer,lda,euclidean,0.178620,-0.019521,-0.052870,-0.055864
6,128,7,solution_all,0.15,True,CountVectorizer,lda,euclidean,0.148488,0.000909,-0.012068,-0.070997
7,1516,5,solution_all,0.40,True,NCutVectorizer,nmf,euclidean,0.065415,-0.004762,-0.001735,-0.072193
8,409,8,solution_all,0.40,True,CountVectorizer,lda,euclidean,0.176273,0.001428,0.001428,-0.075124
9,7,8,solution_all,0.05,True,CountVectorizer,nmf,euclidean,0.152309,-0.014297,-0.080432,-0.075179


In [331]:
c5_df[c5_df['experiment_id'] == 20]

,experiment_id,k,dataset,min_df,is_binary,vectorizer,method,distance,silhouette,coherence_med5,coherence_med10,coherence_med15
31,20,14,solution_all,0.05,True,CountVectorizer,nmf,euclidean,0.130942,0.002497,-0.017493,-0.030117


In [28]:
import math
from itertools import combinations
TOP = 0.04
total = c5_df.shape[0]
rows = math.ceil(total * TOP)
print(rows)
ids = {}
# ids['sil'] = set(sil_df['experiment_id'].head(rows).values.tolist())
ids['c5'] = set(c5_df['experiment_id'].head(rows).values.tolist())
ids['c10'] = set(c10_df['experiment_id'].head(rows).values.tolist())
ids['c15'] = set(c15_df['experiment_id'].head(rows).values.tolist())
# print(len(ids['sil']))
combs = combinations(ids.keys(), 2)
for comb in combs:
    print("Merge set for %s and %s" % comb)
    merge_ids = ids[comb[0]] & ids[comb[1]]
    print("Size: %d" % len(merge_ids))
    print("Ids: %s" % merge_ids)
    print()
    
combs = combinations(ids.keys(), 3)
for comb in combs:
    print("Merge set for %s, %s and %s" % comb)
    merge_ids = ids[comb[0]] & ids[comb[1]] & ids[comb[2]]
    print("Size: %d" % len(merge_ids))
    print("Ids: %s" % merge_ids)
    print()

42
Merge set for c5 and c10
Size: 8
Ids: {1527, 489, 1136, 22, 1463, 409, 349, 350}

Merge set for c5 and c15
Size: 5
Ids: {10, 1517, 1463, 409, 63}

Merge set for c10 and c15
Size: 16
Ids: {128, 293, 358, 8, 296, 1516, 397, 301, 16, 401, 245, 246, 1463, 408, 409, 123}

Merge set for c5, c10 and c15
Size: 2
Ids: {409, 1463}



In [92]:
from tqdm import tqdm_notebook
import time

for i in tqdm_notebook(range(3)):
    for j in tqdm_notebook(range(3), desc='oi'):
        time.sleep(2)
    print(end='\r')